# Multimodal Image and Text Classification Project

## Overview

This project aims to develop a deep learning model that classifies various types of garbage into predefined categories using a combination of image and text data. The architecture combines a Convolutional Neural Network (CNN) for image feature extraction and a DistilBERT model for text feature extraction.

The project uses `PyTorch` as the deep learning framework, `transformers` for text processing, and is designed to run efficiently on a GPU cluster using Slurm.

## Project Structure
Data is expected be stored in the following directory or under `/work/TALC/enel645_2024f/garbage_data`

```
├── data/                            # Dataset folder
│   ├── CVPR_2024_dataset_Train      # Training images
│   ├── CVPR_2024_dataset_Val        # Validation images
│   └── CVPR_2024_dataset_Test       # Test images
...
├── setup_conda_environment.sh       # Script to set up the conda environment
├── model_notebook.py                # Jupyter notebook for training and evaluating the model
├── train_model.py                   # Python script for training the model (same code as the notebook)
├── slurm_job_gpu.sh                 # Slurm submission script (using GPU)
├── slurm_job_cpu.sh                 # Slurm submission script (using CPU)
...
└── temp/                            # Folder containing temporary code that was tried. There is no guarantee that this code will work but it   gives an overview of the motheds I tried. It also contains 3 CSV files (train_image_descriptions, val_image_descriptions, test_image_descriptions) which contain captions that were generated using Microsoft's Frorence model. 

```

## Setup Instructions

### Prerequisites

- [Anaconda/Miniconda](https://docs.conda.io/projects/conda/en/latest/user-guide/install/index.html) installed.
- Access to a GPU-enabled server or cluster (recommended)

### Step 1: Set up Conda Environment

1. **Create and activate the conda environment** using the provided script:

   ```bash
   ./setup_conda_environment.sh
   ```

   This script checks for an existing conda environment named `mm_enel645_assg2`, creates it if it does not exist, and installs all necessary packages.
   Alternatively, if you have all the required libraries already installed, you can skip this step. 
2. **Verify the installation**:

   ```bash
   conda activate mm_enel645_assg2
   conda list
   ```

### Step 2: Train the Model

1. **Submit the training job to the cluster** using the provided Slurm script:

   ```bash
   sbatch slurm_job_gpu.sh
   ```

   This Slurm script (`slurm_job_gpu.sh`) allocates the necessary resources (nodes, CPUs, GPUs, memory, etc.) and runs the `train_model.py` script to start training the model.

2. **Monitor the job**:

   You can monitor the Slurm job with:

   ```bash
   squeue -u your_username
   ```

   You can also check the output and error logs using the `.out` and `.err` files generated by Slurm.

### Step 3: Model Evaluation

After training completes, the model will be evaluated on the test set. The results are displayed in a confusion matrix, along with other performance metrics like accuracy, precision, and recall.

## Files Description

- **`setup_conda_environment.sh`**: Bash script to set up the conda environment and install all dependencies.
- **`train_model.py`**: Python script containing all necessary code to train the multimodal model.
- **`slurm_job_gpu/cpu.sh`**: Slurm batch script for submitting the training job to a cluster.
- **`train_image_descriptions.csv`**: CSV file that contains descriptions/captions for the training images.

## Requirements

- `torch`: For deep learning model creation.
- `torchvision`: For image data processing.
- `transformers`: For text data processing with DistilBERT.
- `scikit-learn`: For evaluation metrics.
- `matplotlib` and `seaborn`: For plotting confusion matrix and other visualizations.
- `pillow` and `numpy`: For image and numerical data handling.

## Example Usage

1. **Run the training script locally**:

   ```bash
   python train_model.py
   ```

2. **Submit the job to a cluster**:

   ```bash
   sbatch slurm_job_gpu.sh
   ```

3. **Check the output** in the Slurm `.out` file to monitor the progress and performance metrics.

## Data and Methods

This project uses a combination of image data and text descriptions to classify images into four categories: Black, Blue, Green, and TTR. The model architecture consists of a ResNet50 model for image feature extraction and a DistilBERT model for processing text descriptions.
A fully connected neural network combines the features and classifies input data into one of the said output classes. 

### Dataset

- **Image Data**: RGB images of varying resolutions.
- **Text Data**: Captions associated with each image.

### Methodology

1. **Data Preprocessing**: Resizing images, tokenizing text, and applying necessary augmentations.
2. **Model**: A multimodal neural network with ResNet50 and DistilBERT components.
3. **Training**: Uses a weighted cross-entropy loss function to account for class imbalance.
4. **Evaluation**: Performance metrics like accuracy, precision, recall, and confusion matrix are computed.


Now the fun part, enjoy!

In [ ]:
# Importing necessary libraries for building and training models, handling data, and visualizations

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from torchvision.models import ResNet50_Weights
from transformers import DistilBertModel, DistilBertTokenizer
from PIL import Image
import numpy as np
import re
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import socket
import random
from collections import Counter

hostname = socket.gethostname()

print("Running on Hostname:", hostname)

# Define device preference order: CUDA if available, MPS if available (for Apple Silicon), otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f'Using device: {device}')


In [ ]:
# Data preparation

# Label map - Converts class names to corresponding integer labels for training purposes
label_map = {"Black": 0, "Blue": 1, "Green": 2, "TTR": 3}

# General function to extract text description from filenames
def extract_text_from_filename(filename):
    file_name_no_ext, _ = os.path.splitext(filename)
    # Remove trailing numbers and replace underscores with spaces
    return re.sub(r'\d+', '', file_name_no_ext.replace('_', ' '))

# Load data - Given a data directory, this function loads images and extracts labels based on folder names
def load_data(data_dir, label_map, file_extensions=('.png', '.jpg', '.jpeg')):
    image_paths, texts, labels = [], [], []

    # Iterate through each class label and corresponding index
    for label_name, label_idx in label_map.items():
        folder_path = os.path.join(data_dir, label_name)
        if not os.path.isdir(folder_path):
            continue

        # Iterate through all files in the class folder
        for filename in os.listdir(folder_path):
            # Select only image files with specified extensions
            if filename.lower().endswith(file_extensions):
                image_path = os.path.join(folder_path, filename)
                image_paths.append(image_path)
                texts.append(extract_text_from_filename(filename))
                labels.append(label_idx)

    # Return loaded image paths, text descriptions, and labels as numpy arrays
    return np.array(image_paths), np.array(texts), np.array(labels)

# Function to detect the base directory based on whether it's running on a local machine or TALC cluster
def detect_base_directory():
    possible_dirs = [
        r"/work/TALC/enel645_2024f/garbage_data",  # Directory on TALC cluster
        r"../../data/enel645_2024f/garbage_data"   # Directory on LAPTOP - relative path
    ] # Add your path here if the data is stored in a different location

    for base_dir in possible_dirs:
        if os.path.exists(base_dir):
            print(f"Using base directory: {base_dir}")
            return base_dir
    # Alternatively, you can use the host name, but it works just fine as is

    # Raise an error if no valid data directory is found
    raise ValueError("No valid base directory found.")

# Define the custom dataset class to handle multimodal data (images + text descriptions)

class MultimodalDataset(Dataset):
    def __init__(self, image_paths, texts, labels, transform=None, tokenizer=None, max_len=24):
        """
        Custom Dataset to handle multimodal inputs (image and text).
        
        Args:
            image_paths (list): List of file paths to the images.
            texts (list): List of text descriptions corresponding to the images.
            labels (list): List of labels corresponding to the images/texts
            transform: A function/transform to apply to the images.
            tokenizer (transformers.PreTrainedTokenizer): Tokenizer for text processing.
            max_len: Maximum length of tokenized text. Defaults to 24.
        """
        self.image_paths = image_paths
        self.texts = texts
        self.labels = labels
        self.transform = transform
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        """Returns the total number of samples in the dataset."""
        return len(self.image_paths)

    def __getitem__(self, idx):
        """
        Retrieves and processes a single sample from the dataset at the given index.
        
        Args:
            idx (int): Index of the sample to retrieve.
        
        Returns:
            dict: A dictionary containing processed image, tokenized text input IDs, attention mask, and label.
        """

        # Load and transform the image (if a transform is provided)
        image = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)

        # Tokenize the text description using the provided tokenizer (DistilBERT)
        text = str(self.texts[idx])
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        # Get the label for the current sample
        label = self.labels[idx]

        # Return a dictionary containing image, input_ids, attention_mask, and label
        return {
            'image': image,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }
    

# Data augmentation and preprocessing transformations for images
def get_image_transforms(image_size=(224, 224)):
    return transforms.Compose([
        transforms.Resize(image_size),          # Resize the image to the specified size (default 224x224)
        transforms.RandomHorizontalFlip(),      # Randomly flip the image horizontally
        transforms.RandomVerticalFlip(),        # Randomly flip the image vertically
        transforms.RandomRotation(20),          # Randomly rotate the image (max 20 degrees)
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Random color jittering
        transforms.ToTensor(),                  # Convert image to PyTorch tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet mean & std
    ])

# Calculate class weights using inverse frequency to handle class imbalance
def calculate_class_weights(train_dir, label_map):
    class_counts = np.zeros(len(label_map), dtype=np.int32)
    for label_name, label_idx in label_map.items():
        folder_path = os.path.join(train_dir, label_name)
        if os.path.isdir(folder_path):
            class_counts[label_idx] = len([f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

    class_weights = 1.0 / class_counts
    class_weights = class_weights / class_weights.sum()  # Normalize (sum to 1)
    return class_weights

# Usage - Get the base directory and start working
base_dir = detect_base_directory()

# Define paths for training, validation, and testing datasets
train_dir = os.path.join(base_dir, "CVPR_2024_dataset_Train")
val_dir = os.path.join(base_dir, "CVPR_2024_dataset_Val")
test_dir = os.path.join(base_dir, "CVPR_2024_dataset_Test")

# Load data for training, validation, and testing
train_image_paths, train_texts, train_labels = load_data(train_dir, label_map)
val_image_paths,   val_texts,   val_labels   = load_data(val_dir, label_map)
test_image_paths,  test_texts,  test_labels  = load_data(test_dir, label_map)

# Get image transformation and text tokenizer
image_transform = get_image_transforms(image_size=(224, 224))
text_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Create datasets
train_dataset = MultimodalDataset(train_image_paths, train_texts, train_labels, transform=image_transform, tokenizer=text_tokenizer)
val_dataset   = MultimodalDataset(val_image_paths, val_texts, val_labels, transform=image_transform, tokenizer=text_tokenizer)
test_dataset  = MultimodalDataset(test_image_paths, test_texts, test_labels, transform=image_transform, tokenizer=text_tokenizer)

# Create DataLoaders with multi-threaded loading

# Adjustable based on available memory and system capabilities. 
# For local training, any value higher than 16 will cause an out of memory error. For the cluster, higher values can be used
batch_size = 8
if "TALC" in hostname:
    batch_size = 64

nworkers = os.cpu_count()//2 # for some reason this crashes the DataLoaders on apple sillicon, set it to 0
nworkers = 0 
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=nworkers, pin_memory=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=nworkers, pin_memory=True)
test_loader  = DataLoader(test_dataset, batch_size=2*batch_size, shuffle=False, num_workers=nworkers, pin_memory=True) # This does not change the time but it might reduce the overhead

# Calculate class weights for loss function and move to device
class_weights = torch.tensor((np.array([1.5, 1, 1, 1.5]) * calculate_class_weights(train_dir, label_map) * 4/5), dtype=torch.float32).to(device)

# Print class weights
for class_name, weight in zip(label_map.keys(), class_weights):
    print(f'{class_name} weight: {weight:.4f}')

# Plotting class distribution and weights for visualization
class_names = list(label_map.keys())
class_counts = np.array([len(os.listdir(os.path.join(train_dir, name))) for name in class_names])

fig, ax1 = plt.subplots(figsize=(10, 6))

# Bar plot for class counts
color = 'tab:blue'
ax1.set_xlabel('Class')
ax1.set_ylabel('Number of Samples', color=color)
ax1.bar(class_names, class_counts, color=color, alpha=0.6)
ax1.tick_params(axis='y', labelcolor=color)

# Create a second y-axis for class weights
ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Class Weights', color=color)
ax2.plot(class_names, class_weights.cpu().numpy(), color=color, marker='o', linestyle='-', linewidth=2, markersize=8)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_ylim(0, 1)

# Add plot title
plt.title('Class Distribution and Weights in the Training Set')

# Show plot
plt.tight_layout()
plt.show()

# As seen in the following plot, there is a significant imbalance in class distributions, 
# as shown by the Blue class having almost double the number of samples compared to other classes, 
# using class weights is crucial to prevent the model from being biased towards the majority class.


In [ ]:

# Define the multimodal model combining ResNet50 (image) and DistilBERT (text)
class MultimodalModel(nn.Module):
    def __init__(self, num_classes):
        super(MultimodalModel, self).__init__()
        # Load pretrained ResNet50 model
        self.resnet = models.resnet50(weights=ResNet50_Weights.DEFAULT)
        self.resnet.fc = nn.Identity()  # Remove the classification layer of ResNet50
        self.resnet_feature_dim = 2048  # ResNet50 output feature dimension

        # Load DistilBERT model
        self.distilbert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.distilbert_feature_dim = 768  # DistilBERT output feature dimension - self.distilbert.config.hidden_size

        # Fully connected layers for combining features
        self.fc = nn.Sequential(
            nn.Linear(self.resnet_feature_dim + self.distilbert_feature_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.3),  # 30% dropout to control overfitting
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, images, input_ids, attention_mask):
        # Extract image features using ResNet50
        image_features = self.resnet(images)

        # Extract text features using DistilBERT
        text_features = self.distilbert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]

        # Concatenate image and text features
        combined_features = torch.cat((image_features, text_features), dim=1)

        # Pass through the fully connected layers
        output = self.fc(combined_features)
        return output

# Hyperparameters and model setup
num_classes = len(label_map)
learning_rate = 2e-5

# Instantiate model and move to device
mmodel = MultimodalModel(num_classes=num_classes).to(device)

# Define loss function with class weights to handle class imbalance
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Define optimizer (AdamW with weight decay)
optimizer = optim.AdamW(mmodel.parameters(), lr=learning_rate)

# Define learning rate scheduler with exponential decay
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

In [ ]:
best_val_loss = float('inf')    # Start with a very high value (infinity)
patience = 5                    # How many epochs without improvement to allow
num_epochs = 10                 # Maximum number of training epochs
epochs_no_improve = 0           # Counter for epochs with no improvement

train_losses = []               # Array to store training losses
val_losses = []                 # Array to store validation losses

print("Starting model training. Hang tight, that might take a while!")
for epoch in range(num_epochs):
    # Training phase
    mmodel.train()
    train_loss = 0.0  # Reset value
    for batch in train_loader:
        # Move batch data to device
        images = batch['image'].to(device, non_blocking=True)
        input_ids = batch['input_ids'].to(device, non_blocking=True)
        attention_mask = batch['attention_mask'].to(device, non_blocking=True)
        labels = batch['label'].to(device, non_blocking=True)

        # Reset gradients
        optimizer.zero_grad()

        # Forward pass: Calculate model predictions for the current batch
        outputs = mmodel(images, input_ids, attention_mask)

        # Calculate the cross-entropy loss between predictions and true labels
        loss = criterion(outputs, labels)

        # Backward pass to calculate gradients with respect to the model parameters
        loss.backward()

        # Update the model parameters using the computed gradients
        optimizer.step()

        # Accumulate training loss
        train_loss += loss.item()
    
    # Normalize train_loss by the number of batches and store it
    train_loss /= len(train_loader)
    train_losses.append(train_loss)  # Save train loss for the epoch
    
    # Validation phase
    mmodel.eval()  # Change state to validation mode
    val_loss = 0.0
    y_true, y_pred = [], []
    with torch.no_grad():  # Stop calculating gradients during validation
        for batch in val_loader:
            # Move batch data to device
            images = batch['image'].to(device, non_blocking=True)
            input_ids = batch['input_ids'].to(device, non_blocking=True)
            attention_mask = batch['attention_mask'].to(device, non_blocking=True)
            labels = batch['label'].to(device, non_blocking=True)

            # Forward pass: Calculate model predictions for the current batch
            outputs = mmodel(images, input_ids, attention_mask)
            
            # Calculate validation loss
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Get the index of the maximum log-probability as the prediction
            _, preds = torch.max(outputs, dim=1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    # Normalize the validation loss by the number of batches and store it
    val_loss /= len(val_loader)
    val_losses.append(val_loss)  # Save validation loss for the epoch

    # Calculate statistical metrics on the entire validation set
    val_accuracy = accuracy_score(y_true, y_pred)
    val_precision = precision_score(y_true, y_pred, average='weighted')
    val_recall = recall_score(y_true, y_pred, average='weighted')
    print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, '
          f'Val Acc: {val_accuracy:.4f}, Val Prec: {val_precision:.4f}, Val Recall: {val_recall:.4f}')

    # Update the learning rate (exponential decay, gamma=0.9)
    scheduler.step()

    # Check for early stopping
    if val_loss < best_val_loss:
        print(f"Validation loss improved ({best_val_loss:.4f} → {val_loss:.4f}). Saving model...")
        torch.save(mmodel.state_dict(), 'best_multimodal_model.pth')  # Save the best model with epoch info
        best_val_loss = val_loss  # Update best validation loss value
        epochs_no_improve = 0  # Reset the counter
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("Early stopping triggered. Restoring the best model...")
            mmodel.load_state_dict(torch.load('best_multimodal_model.pth'))  # Restore best model
            break


In [ ]:
# Plot the training and validation losses and investigate them

fig, ax1 = plt.subplots(figsize=(10, 5))

# First y-axis for training loss
ax1.plot(train_losses, color='cornflowerblue', label='Log Training Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Training Loss', color='cornflowerblue')
ax1.tick_params(axis='y', labelcolor='cornflowerblue')

# Create a second y-axis for validation loss
ax2 = ax1.twinx()
ax2.plot(val_losses, color='salmon', label='Log Validation Loss')
ax2.set_ylabel('Validation Loss', color='salmon')
ax2.tick_params(axis='y', labelcolor='salmon')

# Add a title and show the plot
plt.title('Training and Validation Loss Over Epochs')
fig.tight_layout()
plt.show()

In [ ]:
# Load the model and evaluate on the test set. Steps are similar to the validation phase
# Before running this cell, make sure that the '# DL Model' cell was properly executed

print("Evaluating the model on the test dataset.")

# Load the best model weights
mmodel.load_state_dict(torch.load('best_multimodal_model.pth', map_location=device, weights_only=True))
mmodel.eval()  # Set the model to evaluation mode

# Initialize variables for collecting results
test_predictions = []
test_labels = []

# Track progress
num_batches = len(test_loader)
print_every = max(1, num_batches // 10)  # Print every 5% of batches, at minimum every 1 batch

# Disable gradient computation for inference
with torch.no_grad():
    for batch_idx, batch in enumerate(test_loader):
        # Print feedback on progress
        if (batch_idx + 1) % print_every == 0 or (batch_idx + 1) == num_batches:
            print(f"Processing batch {batch_idx + 1}/{num_batches}")

        # Move batch data to the device
        images = batch['image'].to(device)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # Get model predictions
        outputs = mmodel(images, input_ids, attention_mask)
        _, preds = torch.max(outputs, dim=1)

        # Store predictions and true labels for metric calculation
        test_predictions.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())
        

print("Evaluation on test dataset complete.")


In [ ]:
# Statistical analysis of the test set
# Make sure the '# Data preparation' cell was executed before running this cell

# Confusion matrix calculation
cm = confusion_matrix(test_labels, test_predictions)

# Normalize the confusion matrix by the number of actual samples in each class (row-wise normalization)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Plotting the normalized confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_normalized, annot=True, cmap='Blues', fmt='.2f', cbar=False, xticklabels=label_map.keys(), yticklabels=label_map.keys())
plt.title('Normalized Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Calculate overall test set metrics
test_accuracy = accuracy_score(test_labels, test_predictions)
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')  # Calculate weighted F1 score

print(f'Test Accuracy: {test_accuracy:.4f}, Test Precision: {test_precision:.4f}, Test Recall: {test_recall:.4f}, Test F1 Score: {test_f1:.4f}')

# Calculate the accuracy for each class
class_accuracy = cm.diagonal() / cm.sum(axis=1)

# Map label indices back to class names
label_names = {v: k for k, v in label_map.items()}

# Print accuracy per class
for i, accuracy in enumerate(class_accuracy):
    class_name = label_names[i]
    print(f'Accuracy for {class_name}: {accuracy:.4f}')


# Looking at the CM, we can see that the model excels at detecting 'Green' samples (high sensitivity with low false detections).
# It also does good job in the 'Blue' outcomes, but the false detections are quite high for this class (look at the second column from the left).
# This suggests that the model has some trouble with this class as many samples from the Black and TTR classes are misclassifed as Blue. 
# We'll see that later in the code also.

In [ ]:
# Take a loot at some of the miscalssifed results

i0 = 3  # True label
i1 = 1  # Predicted label (false predictions)
mistaken_indices = [i for i, (true, pred) in enumerate(zip(test_labels, test_predictions)) if true == i0 and pred == i1]

# Display the count and indices of misclassified cases
print(f'Number of cases where {label_names[i0]} was mistakenly identified as {label_names[i1]}: {len(mistaken_indices)}')
print("Indices of the mistakes:", mistaken_indices)

# Randomly select up to 10 misclassified images if there are enough examples
max_display = 10
if len(mistaken_indices) > max_display:
    display_indices = random.sample(mistaken_indices, max_display)
else:
    display_indices = mistaken_indices

# Loop through the selected mistaken indices and display the images
for ind in display_indices:
    image_path = test_image_paths[ind]  # Get the image path for the current mistaken index
    image = Image.open(image_path)      # Load the image
    
    # Plot the image with a smaller figure size
    plt.figure(figsize=(4, 3))
    plt.imshow(image)
    plt.axis('off')  # Hide the axes
    plt.title(f"True: {label_names[i0]}, Predicted: {label_names[i1]}\nImage Path: {image_path}")
    plt.show()
    
    # Print the corresponding text description if available
    print(f"Text description: {test_texts[ind]}")


In [ ]:
# Now let's take a loot at the misclassified cases and see if there is a common ground for them

all_misclassified_texts = []

for i0 in range(4):  # assuming there are 4 classes, indexed from 0 to 3
    
    # Fixed predicted label for analysis; adjust if you want to analyze multiple predicted labels
    # i1 = 1 : Blue 
    i1 = 1
    if i0 == i1:
        continue

    # Find misclassified indices where the true label is i0 and the predicted label is i1
    mistaken_indices = [i for i, (true, pred) in enumerate(zip(test_labels, test_predictions)) if true == i0 and pred == i1]
    
    # Append misclassified texts for the current i0 value
    all_misclassified_texts.extend([test_texts[ind] for ind in mistaken_indices])

# Combine all texts into a single string
combined_text = " ".join(all_misclassified_texts).lower()

# Remove punctuation and split into individual words
words = re.findall(r'\b\w+\b', combined_text)

# Count word frequencies
word_counts = Counter(words)

# Display the 10 most common words
print("Most common words in all misclassified texts:")
for word, count in word_counts.most_common(10):
    print(f"{word}: {count/len(all_misclassified_texts)*100:.1f}%")

# In the current run, I see the following results (i1=1, Blue):
''' Most common words in all misclassified texts:
empty: 25.0%
bottle: 16.8%
container: 15.2%
plastic: 13.0%
bag: 11.4%
paper: 6.5%
box: 6.0%
cleaner: 5.4%
foil: 4.3%
old: 3.8% '''

# This means that cases that were miscalssified as Blue does contain some common ground. 
# For example, the words 'empty', 'bottle', and 'container' appear in many of the cases. 
# This is the test set therefore it's not recomended to go back and modify the model but it can give us some directions for improvement. 
# One approach that I was thinking of is to create an additional classifer for 'Blue' only. Once we get a 'Blue' prediction, 
# we apply this classifier (simple CNN+FCC or using a caption generator to enhace the description) to get a better outcome 
# (to distingiush between 'True Blue' and the other classes)